# Introduction to Building LLM Agents with Tools and Tracing

<!--- @wandbcode{bedrock-webinar-July-2025} -->

This script walks through the process of building a simple LLM-powered agent that can use tools (functions) to answer questions. We'll cover:
1. Making basic LLM calls.
2. Introducing Weave for tracing and observability.
3. Defining tools for the LLM (manually and automatically).
4. Implementing a basic agentic loop.
5. Structuring the agent using Python classes.
6. Running the agent on a multi-step task.

**Prerequisites:**
Make sure you have the necessary libraries installed:
```bash
!pip install weave openai
```


In [2]:
# Global Configuration & Setup
import os
import inspect
import json
import weave # Must import weave before litellm for auto-patching
import openai
from enum import Enum
from pydantic import BaseModel, Field
from rich.panel import Panel
from rich.markdown import Markdown
from rich.console import Console as RichConsole
from exa_py import Exa
from typing import Any, Callable, Dict, List, get_type_hints

Define a model to use, as we are going to use tool calling you need a capable model like `mistral-large`

In [3]:
class Console(RichConsole):
    def md(self, text): 
        return self.print(Markdown(text))

console = Console()

MODEL_SMALL = "Qwen/Qwen3-235B-A22B-Instruct-2507"
MODEL_MEDIUM = "zai-org/GLM-4.5"
MODEL_LARGE = "moonshotai/Kimi-K2-Instruct"

WANDB_ENTITY = "wandb-applied-ai-team"
WANDB_PROJECT = "london-workshop-2025"

oai_client = openai.OpenAI(
    base_url='https://api.inference.wandb.ai/v1',
    api_key=os.getenv("WANDB_API_KEY"),
    project=f"{WANDB_ENTITY}/{WANDB_PROJECT}")
exa_client = Exa(api_key=os.getenv("EXA_API_KEY"))

Let's log to [W&B Weave](https://weave-docs.wandb.ai/). Weights & Biases (W&B) Weave is a framework for tracking, experimenting with, evaluating, deploying, and improving LLM-based applications. Designed for flexibility and scalability, Weave supports every stage of your LLM application development workflow:

- Tracing & Monitoring: Track LLM calls and application logic to debug and analyze production systems.
- Systematic Iteration: Refine and iterate on prompts, datasets, and models.
- Experimentation: Experiment with different models and prompts in the LLM Playground.
- Evaluation: Use custom or pre-built scorers alongside our comparison tools to systematically assess and enhance application performance.
- Guardrails: Protect your application with pre- and post-safeguards for content moderation, prompt safety, and more.

In [5]:

# Initialize a Weave project. Traces will be sent here.
# You can view them in the Weave UI (usually runs locally).
weave.init(f"{WANDB_ENTITY}/{WANDB_PROJECT}")

/Users/tcapelle/work/deep-research-bot/.venv/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

weave: Logged in as Weights & Biases user: capecape.
weave: View Weave data at https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/weave


## 1. Basic LLM Call with OpenAI SDK

Let's start with a simple call to the LLM using/

![](images/01_trace.png)

In [6]:
# Define a simple message list (conversation history)
messages = [{"role": "user", "content": "Hello, LLM! How does an AI agent work?"}]

# Make the call
response = resp = oai_client.chat.completions.create(
    model = MODEL_SMALL,
    messages=messages,
)
# Print the response content
assistant_response = response.choices[0].message.content
console.md(f"LLM Response:\\n{assistant_response}")

# Click on the 🍩 linke below to see the trace in Weave 👇

weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199ed97-03dc-7daa-9b8c-44e32ba12eed


LLM Response:\nHello! Great question! 😊                                                                           

An AI agent is a software system that perceives its environment, makes decisions, and takes actions to achieve     
specific goals. Think of it like a digital "assistant" that can operate autonomously or semi-autonomously.         

Here’s how an AI agent works, broken down into key components:                                                     

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                   1. Perception                                                   

The agent gathers information from its environment using sensors or data inputs. This could be:                    

 • Text input (like your message to me)                                                                            
 • Camera images (for a robot)                                                                                     
 • Sensor readings (temperature, GPS, etc.)                                                                        
 • Web data, user behavior, etc.                                                                                   

▌ Example: A self-driving car uses cameras and radar to "see" the road.                                          

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                      2. Decision-Making (Reasoning/Planning)                                      

Using the perceived data, the agent processes information and decides what to do. This is where AI techniques come 
in:                                                                                                                

 • Rule-based logic (if-then rules)                                                                                
 • Machine learning models (like neural networks)                                                                  
 • Search algorithms (e.g., finding the best move in a game)                                                       
 • Large Language Models (LLMs) (like me!) for understanding and generating text                                   

▌ Example: A chatbot like me analyzes your question and generates a relevant response.                           

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                     3. Action                                                     

The agent performs actions based on its decisions. These actions affect the environment:                           

 • Moving a robot arm                                                                                              
 • Displaying a response                                                                                           
 • Sending an email                                                                                                
 • Adjusting a thermostat                                                                                          

▌ Example: After understanding your question, I reply with this explanation.                                     

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                        4. Learning (Optional but Powerful)                                        

Some agents improve over time by learning from experience:                                                         

 • Reinforcement learning: learns by trial and error with rewards                                                  
 • Supervised learning: learns from labeled data                                                                   
 • Self-correction: using feedback to improve 

weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199ed97-305b-736a-8ce9-86c99a09c2f7
weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199ed97-52d0-7371-9da2-b557eb569051
weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199ed97-700c-7a8e-907e-ba920e17e8b0
weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199ed97-798d-773a-bd52-7bbd4f4cbded
weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199ed97-8877-7163-a30d-bacc670e8fb5
weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199ed97-cb37-7373-8b28-b46a4cd120aa
weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199ed9a-35fc-7cf8-b9d4-bf47ad3c6e58


Because we imported `weave` and called `weave.init()`, the OpenAI SDK call above was automatically traced. You can open your Weave dashboard and see the trace, including the input messages, output response, latency, model used, etc. This is invaluable for debugging and monitoring.

In [ ]:
# most of the time you would want to define your own operations to trace, for instance to call the model.
# You just need to add the @weave.op decorator to the function and it will be traced.

@weave.op
def call_model(model_name: str, messages: List[Dict[str, Any]], **kwargs) -> str:
    "Call a model with the given messages and kwargs."
    response = oai_client.chat.completions.create(
        model=model_name,
        messages=messages,
        **kwargs
    )

    return response.choices[0].message

response = call_model(model_name=MODEL_SMALL, messages=messages)
console.md(response.content)

Hello! Great question! 😊                                                                                          

An AI agent is a software system that perceives its environment, makes decisions, and takes actions to achieve     
specific goals. Think of it like a digital "assistant" that can operate autonomously or semi-autonomously. Here's a
breakdown of how an AI agent works:                                                                                

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                 🔍 1. Perception                                                  

The agent gathers information from its environment using:                                                          

 • Sensors (in robotics)                                                                                           
 • User inputs (like text or voice)                                                                                
 • APIs, databases, or web scraping                                                                                
 • Predefined data feeds                                                                                           

For example, a chatbot "perceives" your message as input.                                                          

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                        🧠 2. Processing & Decision-Making                                         

This is where the AI "thinks." The agent uses:                                                                     

 • Rules-based logic (if-then rules)                                                                               
 • Machine learning models (like neural networks)                                                                  
 • Large Language Models (LLMs) (e.g., me!)                                                                        
 • Reinforcement learning (learning by trial and reward)                                                           

Based on what it perceives and its internal model, the agent decides what to do next.                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                   🤖 3. Action                                                    

The agent acts on the environment. Actions can include:                                                            

 • Sending a message (like this one!)                                                                              
 • Moving a robot arm                                                                                              
 • Adjusting a thermostat (in smart homes)                                                                         
 • Making a trade in a financial system                                                                            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                           🎯 4. Goal-Oriented Behavior                                            

AI agents are typically designed to achieve goals. For example:                                                    

 • A personal assistant agent might book a meeting.                                                                
 • A self-driving car aims to reach a destination safely.                                                          
 • A customer support bot tries to resolve your query.                                                             

They may use planning, learning, and feedback to improve over time.                                                

────────────────────────────────────────────

weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199ed97-bf87-7b06-b2a7-37e4e1985f1f


![](images/02_nested_trace.png)


## 2. Introducing Tool Calling

Agents become much more powerful when they can use **tools** – external functions or APIs – to get information or perform actions beyond the LLM's internal knowledge. To allow an LLM to use a tool, we need to provide it with a description (schema) of the tool, including its name, purpose, and expected arguments.

Check the Mistral docs for function calling: https://platform.openai.com/docs/guides/function-calling

![](images/function-calling-diagram-steps.png)

First, let's define a simple Python function we want the LLM to be able to call. We add `@weave.op` to trace when this function actually gets executed.


In [8]:
@weave.op 
def add_numbers(a: int, b: int) -> int:
    """Use this tool to add numbers.
    Args:
        a: The first number.
        b: The second number.
    """
    return a + b

In [9]:
add_numbers(1, 2)

3

weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199ed97-52d6-7293-bae4-58ade8165cc8


In [10]:
# this doesn't work...
call_model(model_name=MODEL_SMALL, messages=messages, tools=[add_numbers])

TypeError: Object of type function is not JSON serializable

weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199ed98-1ac3-7de3-90bc-96ac80992aa3
weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199ed98-d8dc-7b0c-9422-2e66f31a3c46


> We need to manually create the JSON schema describing this tool in a format that models *Mistral* understand.

In [11]:
# Manually define the tool schema
tool_add_numbers_schema = {
    "type": "function",
    "function": {
        "name": "add_numbers",
        "description": "Adds two numbers.",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "integer",
                    "description": "The first number."
                },
                "b": {
                    "type": "integer",
                    "description": "The second number."
                }
            },
            "required": ["a", "b"]
        }
    }
}

Now, we make an LLM call, passing the `tools` parameter with our schema. We ask a question that should trigger the tool.

In [12]:
messages = [
    {"role": "system", "content": "You are a helpful assistant use tools to answer questions."},
    {"role": "user", "content": "My lucky numbers are 77 and 11. What is their sum?"}]
response = call_model(model_name=MODEL_SMALL, messages=messages, tools=[tool_add_numbers_schema])
console.print(response)

ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=None,
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageFunctionToolCall(
            id='chatcmpl-tool-f321fc90291f4f68935ee5def283e915',
            function=Function(arguments='{"a": 77, "b": 11}', name='add_numbers'),
            type='function'
        )
    ],
    reasoning_content=None
)

## Manual Tool Call
The LLM's response might contain a request to call our tool (`response.choices[0].message.tool_calls`) or it might respond directly (`response.choices[0].message.content`). If it requests a tool call, we need to:

1. Parse the arguments it provides.
2. Execute our actual Python function (`add_numbers`) with those arguments.
3. (In a real agent loop) Send the result back to the LLM in a new message with `role="tool"`.

Let's manually call the tools in the response.

In [13]:
if response.tool_calls:
    console.print("LLM requested a tool call:")
    for tool_call in response.tool_calls:
        function_name = tool_call.function.name
        function_args_str = tool_call.function.arguments
        function_args = json.loads(function_args_str)
        console.print(f"  - Tool: {function_name}, Args: {function_args_str}")
        if function_name == "add_numbers":
            tool_result_content = add_numbers(**function_args)

console.print(f"Final Result: {tool_result_content}")

LLM requested a tool call:

- Tool: add_numbers, Args: {"a": 77, "b": 11}

Final Result: 88

We need to add the tool call result to the messages (there is actually 2 messages to add)
- the response from the assistant that decided to call the tool
- the tool output

In [14]:
messages.append(response.model_dump())

In [15]:
messages.append({
    "tool_call_id": tool_call.id,
    "role": "tool",
    "name": function_name,
    "content": str(tool_result_content)
})

In [16]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant use tools to answer questions.'},
 {'role': 'user',
  'content': 'My lucky numbers are 77 and 11. What is their sum?'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'annotations': None,
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'chatcmpl-tool-f321fc90291f4f68935ee5def283e915',
    'function': {'arguments': '{"a": 77, "b": 11}', 'name': 'add_numbers'},
    'type': 'function'}],
  'reasoning_content': None},
 {'tool_call_id': 'chatcmpl-tool-f321fc90291f4f68935ee5def283e915',
  'role': 'tool',
  'name': 'add_numbers',
  'content': '88'}]

You should have a sequence of messages like this:


In [17]:
[m["role"] for m in messages]

['system', 'user', 'assistant', 'tool']

Now call the model again with the new messages and it will use the tool call result to answer the question

In [18]:
final_response = call_model(model_name=MODEL_SMALL, messages=messages)
console.print(final_response.content)

The sum of your lucky numbers, 77 and 11, is 88.

## 3. Simplifying Tool Definition with a Processor Function

Manually writing JSON schemas is tedious and error-prone. We can automate this by inspecting our Python function's signature, type hints, and docstring.

First, let's define a helper function (`generate_tool_schema`) that takes a Python function and generates the schema.


In [19]:
def generate_tool_schema(func: Callable) -> dict:
    """Given a Python function, generate a tool-compatible JSON schema.
    Handles basic types and Enums. Assumes docstrings are formatted for arg descriptions.
    """
    signature = inspect.signature(func)
    parameters = signature.parameters
    type_hints = get_type_hints(func)

    schema = {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": inspect.getdoc(func).split("\\n")[0] if inspect.getdoc(func) else "",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    }

    docstring = inspect.getdoc(func)
    param_descriptions = {}
    if docstring:
        args_section = False
        current_param = None
        for line in docstring.split('\\n'):
            line_stripped = line.strip()
            if line_stripped.lower().startswith(("args:", "arguments:", "parameters:")):
                args_section = True
                continue
            if args_section:
                if ":" in line_stripped:
                    param_name, desc = line_stripped.split(":", 1)
                    param_descriptions[param_name.strip()] = desc.strip()
                elif line_stripped and not line_stripped.startswith(" "): # Heuristic: end of args section
                     args_section = False

    for name, param in parameters.items():
        is_required = param.default == inspect.Parameter.empty
        param_type = type_hints.get(name, Any)
        json_type = "string"
        param_schema = {}

        # Basic type mapping
        if param_type == str: json_type = "string"
        elif param_type == int: json_type = "integer"
        elif param_type == float: json_type = "number"
        elif param_type == bool: json_type = "boolean"
        elif hasattr(param_type, '__origin__') and param_type.__origin__ is list: # Handle List[type]
             item_type = param_type.__args__[0] if param_type.__args__ else Any
             if item_type == str: param_schema = {"type": "array", "items": {"type": "string"}}
             elif item_type == int: param_schema = {"type": "array", "items": {"type": "integer"}}
             # Add more list item types if needed
             else: param_schema = {"type": "array", "items": {"type": "string"}} # Default list item type
        elif hasattr(param_type, "__members__") and issubclass(param_type, Enum): # Handle Enum
             json_type = "string"
             param_schema["enum"] = [e.value for e in param_type]

        if not param_schema: # If not set by List or Enum
            param_schema["type"] = json_type

        param_schema["description"] = param_descriptions.get(name, "")

        if param.default != inspect.Parameter.empty and param.default is not None:
             param_schema["default"] = param.default # Note: OpenAI schema doesn't officially use default, but useful metadata

        schema["function"]["parameters"]["properties"][name] = param_schema
        if is_required:
            schema["function"]["parameters"]["required"].append(name)
    return schema

Now we can use this function to automatically generate the schema for our tool.

In [20]:
tool_schema = generate_tool_schema(add_numbers)
console.print(tool_schema)

{
    'type': 'function',
    'function': {
        'name': 'add_numbers',
        'description': 'Use this tool to add numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
        'parameters': {
            'type': 'object',
            'properties': {
                'a': {'type': 'integer', 'description': ''},
                'b': {'type': 'integer', 'description': ''}
            },
            'required': ['a', 'b']
        }
    }
}

Now, we define a `function_tool` "processor". This isn't a decorator in the `@` syntax sense here, but a function that we call *after* defining our tool function. It uses `generate_tool_schema` to attach the schema to the function object itself.


In [21]:
def function_tool(func: Callable) -> Callable:
    """Attaches a tool schema to the function and marks it as a tool.
    Call this *after* defining your function: my_func = function_tool(my_func)
    """
    try:
        func.tool_schema = generate_tool_schema(func)
        func.is_tool = True # Mark it as a tool
    except Exception as e:
        console.print(f"Error processing tool {func.__name__}: {e}")
        # Optionally raise or mark as failed
        func.tool_schema = None
        func.is_tool = False
    return func

We can use this function to automatically generate the schema for our tool, as a decorator or after the function is defined.

In [22]:
add_numbers = function_tool(add_numbers)
console.print(add_numbers.tool_schema)


{
    'type': 'function',
    'function': {
        'name': 'add_numbers',
        'description': 'Use this tool to add numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
        'parameters': {
            'type': 'object',
            'properties': {
                'a': {'type': 'integer', 'description': ''},
                'b': {'type': 'integer', 'description': ''}
            },
            'required': ['a', 'b']
        }
    }
}

In [23]:
add_numbers.tool_schema

{'type': 'function',
 'function': {'name': 'add_numbers',
  'description': 'Use this tool to add numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'integer', 'description': ''},
    'b': {'type': 'integer', 'description': ''}},
   'required': ['a', 'b']}}}

and call the tool =)

In [ ]:
add_numbers(1, 2)

weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199ed97-a14d-7bd8-a7ea-f9d9e369ad24


3

weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199ed98-7745-74bf-9064-b460b906d9ec


### 3.1 Real Example using an API based tool

We are going to use the [EXA search API](https://docs.exa.ai/reference/getting-started).
- How does [EXA search works](https://docs.exa.ai/reference/how-exa-search-works#how-exa-search-works)
- Using exa search [as tool calling](https://docs.exa.ai/reference/openai-tool-calling)

In [25]:
query = "Recipes for cooking seabass?"

search_res = exa_client.search_and_contents(query=query, type='auto', num_results=4)

In [26]:
console.print(search_res)

SearchResponse(
    results=[
        Result(
            url='https://www.facebook.com/chefbillyparisi/posts/the-best-way-to-cook-chilean-sea-bass-that-takes-it
-to-the-next-level/1239156604237561/',
            id='https://www.facebook.com/chefbillyparisi/posts/the-best-way-to-cook-chilean-sea-bass-that-takes-it-
to-the-next-level/1239156604237561/',
            title='The BEST way to Cook Chilean Sea Bass that takes it to the NEXT ...',
            score=None,
            published_date='2025-03-27T00:00:00.000Z',
            author='',
            image='',
            favicon=None,
            subpages=None,
            extras=None,
            text='# [_Facebook_](https://www.facebook.com/)\n\n| | |\n| --- | --- |\n| Email or phone | Password 
|\n| [Forgot account?](https://www.facebook.com/recover/initiate?lwv=100&ars=royal_blue_bar) |\n\n[Create new 
account](https://www.facebook.com/r.php?locale=en_US)\n\n## You’re Temporarily Blocked\n\n## You’re Temporarily 
Blocked\n\nIt looks like you were misusing this feature by going too fast. You’ve been temporarily blocked from 
using it.\n\n[Back](https://www.facebook.com/www.facebook.com)\n\n- English (US)\n- 
[Español](https://www.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-best-
way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- [Français 
(France)](https://es-la.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-bes
t-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- 
[中文(简体)](https://fr-fr.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-
best-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- 
[العربية](https://zh-cn.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-bes
t-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- [Português 
(Brasil)](https://ar-ar.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-bes
t-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- 
[Italiano](https://pt-br.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-be
st-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- 
[한국어](https://it-it.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-best
-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- 
[Deutsch](https://ko-kr.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-bes
t-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- 
[हिन्दी](https://de-de.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-best-wa
y-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- 
[日本語](https://hi-in.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-best
-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n\n- [Sign 
Up](https://www.facebook.com/reg/)\n- [Log In](https://www.facebook.com/login/)\n- 
[Messenger](https://messenger.com/)\n- [Facebook Lite](https://www.facebook.com/lite/)\n- 
[Video](https://www.facebook.com/watch/)\n- [Meta Pay](https://about.meta.com/technologies/meta-pay)\n- [Meta 
Store](https://www.meta.com/)\n- [Meta Quest](https://www.meta.com/quest/)\n- [Ray-Ban 
Meta](https://www.meta.com/smart-glasses/)\n- [Meta AI](https://www.meta.ai/)\n- [Meta AI more 
content](https://www.meta.ai/pages/what-is-labubu/?utm_source=foa_web_footer)\n- 
[Instagram](https://l.facebook.com/l.php?u=https%3A%2F%2Fwww.instagram.com%2F&h=AT0-jEAtr7zT126tLWKVRA4hvnWz4eo9c9M
a3SnO8b6Kcj5yAY6G-X4XWuEItQo8e9xxX2xa-WUiz7YohLnNWuIJHWsqUOCFd44idFwg3em

Let's explore the payload

In [27]:
console.md("\n-------------------\n".join(result.text for result in search_res.results))

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                    ]8;id=462130;https://www.facebook.com/\Facebook]8;;\                                                     ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                              
                              
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  Email or phone    Password  
  ]8;id=688947;https://www.facebook.com/recover/initiate?lwv=100&ars=royal_blue_bar\Forgot account?]8;;\             
                              

]8;id=659022;https://www.facebook.com/r.php?locale=en_US\Create new account]8;;\                                                                                                 


                                            You’re Temporarily Blocked                                             


                                            You’re Temporarily Blocked                                             

It looks like you were misusing this feature by going too fast. You’ve been temporarily blocked from using it.     

]8;id=521083;https://www.facebook.com/www.facebook.com\Back]8;;\                                                                                                               

 • English (US)                                                                                                    
 • ]8;id=743608;https://www.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-best-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F\Español]8;;\                                                                                                         
 • ]8;id=46282;https://es-la.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-best-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F\Français (France)]8;;\                                                                                               
 • ]8;id=251184;https://fr-fr.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-best-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F\中文(简体)]8;;\                                                                                                      
 • ]8;id=438955;https://zh-cn.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-best-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F\العربية]8;;\                                                                                                         
 • ]8;id=863338;https://ar-ar.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-best-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F\Português (Brasil)]8;;\                                                                                              
 • ]8;id=892493;https://pt-br.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-best-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F\Italiano]8;;\                                                                                                        
 • ]8;id=159634;https://it-it.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-best-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F\한국어]8;;\                                                                                                          
 • ]8;id=968416;https://ko-kr.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-best-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F

In [28]:
@weave.op 
@function_tool # <- we can use the decorator to automatically generate the tool schema
def exa_search(query: str, num_results: int = 5) -> list[dict[str, str]]:
    """Perform a search query on the web and retrieve the most relevant URLs and web content.
    
    This function uses the Exa search API to find relevant web pages based on the query
    and returns their titles, text content, and URLs.
    
    Args:
        query: The search query. Use detailed, specific queries for better results.
               The quality of results depends on the specificity of the query.
        num_results: The number of search results to retrieve. Defaults to 5.
    
    Returns:
        A list of dictionaries, each containing:
            - title: The title of the web page
            - text: The text content of the web page
            - url: The URL of the web page
    """
    search_results = exa_client.search_and_contents(query=query, type='auto', num_results=num_results)
    
    output = []
    for result in search_results.results:
        output.append(
            {"title": result.title,
            "text": result.text,
            "url": result.url
            }
        )
    return output

    

In [29]:
exa_search.tool_schema

{'type': 'function',
 'function': {'name': 'exa_search',
  'description': 'Perform a search query on the web and retrieve the most relevant URLs and web content.\n\nThis function uses the Exa search API to find relevant web pages based on the query\nand returns their titles, text content, and URLs.\n\nArgs:\n    query: The search query. Use detailed, specific queries for better results.\n           The quality of results depends on the specificity of the query.\n    num_results: The number of search results to retrieve. Defaults to 5.\n\nReturns:\n    A list of dictionaries, each containing:\n        - title: The title of the web page\n        - text: The text content of the web page\n        - url: The URL of the web page',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string', 'description': ''},
    'num_results': {'type': 'integer', 'description': '', 'default': 5}},
   'required': ['query']}}}

We get a list of results with the relevant metadata.

In [30]:
search_results = exa_search("How do I cook seabass?")
console.print(search_results)

weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199ed97-b954-74e6-addc-a0e8b07d5096


[
    {
        'title': 'Branzino Al Cartoccio Recipe | happybaes.com',
        'text': "[contact](https://happybaes.com/contact) [follow me](https://www.instagram.com/happy_baes/)\n\n# 
Branzino Al Cartoccio Recipe\n\n[Recipes from around the 
world](https://happybaes.com/category/recipes-from-around-the-world/)\n\n## Ingredients\n\nServing size: 2 
adults\xa0+ 2 kids\n\n4 Branzino fillets1 cup of cherry tomatoes, halved1/2 onion, thinly sliced1/2 zucchini, 
thinly sliced1/4 cup of parsley, chopped1 tsp of white cooking wine for each fillet1 lemon, cut into quartersExtra 
virgin olive oilSaltPepper\n\n## Directions\n\n1\\. Preheat your oven to 350F.\n\n2\\. Cut your parchment paper 
into 4 sheets (big enough to fold over the fish) and lay the bottom with sliced zucchini and onions.\n\n3\\. Lay 
your Branzino on top and top the fish with tomatoes, a pinch of salt and pepper, olive oil, and parsley.\n\n4\\. 
Fold the parchment paper in half and seal. Bake for 12 minutes.\n\n5\\. Top with extra basil, squeeze lemon juice 
on top, and 1 tbsp of extra virgin olive oil. Serve and enjoy!\n\n> [View this post on 
Instagram](https://www.instagram.com/reel/DNot33FR2oH/?utm_source=ig_embed&utm_campaign=loading)\n>\n> [A post 
shared by Steph Bae 
(@happy\\_baes)](https://www.instagram.com/reel/DNot33FR2oH/?utm_source=ig_embed&utm_campaign=loading)\n\n### Let's
be _friends_!\n\n### 06",
        'url': 'https://happybaes.com/2025/08/25/branzino-al-cartoccio-recipe/'
    },
    {
        'title': 'Easy Oven Roasted Seabass',
        'text': "### Easy Oven Roasted Seabass\n\nSea Bass is one of my favourite fish, because it is light, the 
flesh is slightly creamy and soft and it takes only a few ingredients as well as very little time to 
prepare.\n\nSea Bass does not have a strong odour, therefore I think it can be suitable even for the pickiest 
seafood eaters. I suggest cooking it only with sea salt and black pepper, a bit of olive oil then finally serving 
it with the highest quality extra virgin olive oil, lemon juice and roasted potatoes or minty mashed peas. Grilled 
veggies like zucchini would go well here, too. Think classics. Everything about this meal is classy. Mediterranean 
style seafood at its best.\n\nMake sure you get the freshest fish possible. When choosing, take a good look: eyes 
should be clear, it should smell fresh and sea-like and its gills should be bright-coloured.\n\nNo reviews yet\n\n#
Easy Oven Roasted Seabass\n\nPrint\n\nMains Quick & Easy 30 min.2 servings\n\nPrint\n\n1 cup\n= 250ml\n\n1 tbsp\n= 
15 ml\n\n1 tsp\n= 5 ml\n\n### Instructions\n\nHeat oven to 180°C (I use setting with a fan). Line a large and 
shallow (important, otherwise the fish will be steamed instead of roasted) cooking tray with baking paper.\n\nIf 
the fish you bought hasn’t been cleaned, cut off the bottom fins from the fish, descale it using a paring knife, 
wash under cold water and cut its belly open. Carefully remove all of the organs, including its gills. I suggest 
you wear gloves for this task so as not to cut yourself.\n\nPat the fish dry, rub it generously with sea salt, 
freshly ground black pepper, some extra virgin olive oil. Place in the oven for 20 minutes, not more.\n\nIn the 
meantime you can boil potatoes and grill some zucchini. Slice the zucchini thinly and arrange in a single layer on 
a very hot & lightly oiled grill pan. Cook on both sides on high heat until it softens and you see brown lines. 
Remove and set aside, repeat with all remaining zucchini slices.\n\nOnce the fish is cooked, fillet it and serve 
with a drizzle of extra virgin olive oil, a generous squeeze of lemon juice, grilled zucchini and cooked potatoes 
or other sides of your choice.\n\nGarnish with fresh dill and black pepper.\n\n#### _Julie’s Tip_\n\nMake sure you 
get the freshest fish possible. When choosing, take a good look: eyes should be clear, it should smell fresh and 
sea-like and its gills should be bright-coloured.\n\n#### Share the Love\n

In [31]:
messages = [
    {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Return in markdown format."},
    {"role": "user", "content": "How do I cook seabass?"}]

response = call_model(model_name=MODEL_SMALL, messages=messages, tools=[exa_search.tool_schema])
console.print(response)

ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=None,
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageFunctionToolCall(
            id='chatcmpl-tool-b6b8a47ec29b4f32a8ed12d652da5ebd',
            function=Function(
                arguments='{"query": "how to cook seabass step by step", "num_results": 5}',
                name='exa_search'
            ),
            type='function'
        )
    ],
    reasoning_content=None
)

Let's create some helper functions to perform the tool calls

In [32]:
from openai.types.chat.chat_completion_message_function_tool_call import ChatCompletionMessageFunctionToolCall

def get_tool(tools: list[Callable], name: str) -> Callable:
    for t in tools:
        if t.__name__ == name:
            return t
    raise KeyError(f"No tool with name {name} found")

def perform_tool_calls(tools: list[Callable], tool_calls: list[ChatCompletionMessageFunctionToolCall]) -> list[dict]:
    "Perform the tool calls and return the messages with the tool call results"
    messages = []
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        with console.status(f"[bold cyan]Executing {function_name}...[/bold cyan]"):
            tool = get_tool(tools, function_name)
            tool_response = tool(**function_args)
        
        # Create panel content
        panel_content = f"[bold cyan]🔧 Tool Call:[/bold cyan] {function_name}\n\n"
        panel_content += f"[dim]Args: {tool_call.function.arguments}[/dim]\n\n"
        
        if isinstance(tool_response, list):
            panel_content += f"[green]✓[/green] Found {len(tool_response)} results"
        else:
            panel_content += f"[green]✓[/green] {function_name} executed successfully"
        
        console.print(Panel(panel_content, border_style="cyan"))
        
        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "content": str(tool_response),
        })
    return messages

In [ ]:

# add the tool call result to the messages
messages.append(response.model_dump())
messages.extend(perform_tool_calls(tools=[exa_search], tool_calls=response.tool_calls))
messages.append({
    "role": "user",
    "content": "Answer my previous query based on the search results.",})

final_response = call_model(model_name=MODEL_SMALL, messages=messages)
console.rule("Final Model Response")
console.md(final_response.content)


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "how to cook seabass step by step", "num_results": 5}                                           │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199ed97-d2f1-7151-bb42-ad9144a02fb8


────────────────────────────────────────────── Final Model Response ───────────────────────────────────────────────

Cooking seabass can be done in several delicious ways, depending on your preference and available time. Here are   
some popular methods based on the search results:                                                                  

                                 1. Pan-Seared Chilean Sea Bass (Quick & Elegant)                                  

This method is ideal for a restaurant-quality dish in under 10–15 minutes.                                         

Steps:                                                                                                             

 • Pat the seabass fillets dry with a paper towel.                                                                 
 • Season one side with salt and pepper.                                                                           
 • Melt butter in a hot non-stick or cast-iron skillet over medium-high heat. Add minced garlic and sauté for 1    
   minute.                                                                                                         
 • Place the fish seasoned-side down and cook for 2–3 minutes until golden. Flip and cook for another 2–3 minutes. 
 • Remove from pan, drizzle with pan juices, and squeeze fresh lemon over the top.                                 

Tip: For best results, sear skin-side down first and avoid moving the fish until it releases easily from the pan.  

Source: ]8;id=446371;https://thebigmansworld.com/chilean-sea-bass-recipe/\The Big Man's World]8;;\                                                                                        

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                  2. Baked Chilean Sea Bass (Simple & Hands-Off)                                   

Perfect for a low-carb, keto-friendly meal.                                                                        

Steps:                                                                                                             

 • Preheat oven to 400°F (200°C).                                                                                  
 • Let the fish come to room temperature for even cooking.                                                         
 • Pat dry, brush with melted butter, and season with salt, pepper, and optional paprika.                          
 • Bake for 12–15 minutes (depending on thickness) until the internal temperature reaches 145°F (63°C).            
 • Finish with a squeeze of lemon juice.                                                                           

Tip: Use evenly sized fillets for consistent cooking.                                                              

Source: ]8;id=510672;https://www.cookedbyjulie.com/baked-chilean-sea-bass/\Cooked by Julie]8;;\                                                                                            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                               3. Pan-Seared with Lemon-Caper Butter (Gourmet Touch)                               

Adds a bright, briny flavor to the rich fish.                                                                      

Steps:                                                                                                             

 • Sear fillets in avocado oil in a hot cast-iron skillet, skin-side down, for 5 minutes.                          
 • Flip, transfer skillet to a 425°F (220°C) oven, and bake for 5 minutes.                                         
 • While cooking, make a sauce: melt butter with lemon juice, capers, and optional white wine.                     
 • Spoon sauce over the fish and garnish with parsley.                                                             

Internal temp: 135–140°F (57–60°C) for optimal flakiness.                                                          

Sou

weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199ed99-7667-75aa-89bf-d63a83e9b614


Let's wrap this in a function:

In [34]:
@weave.op
def research(query: str) -> str:
    messages = [
        {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Return in markdown format."},
        {"role": "user", "content": query}]

    # call model with tools
    response = call_model(
        model_name=MODEL_SMALL, 
        messages=messages, 
        tools=[exa_search.tool_schema])

    # add the response to the messages
    messages.append(response.model_dump())

    # perform the tool calls
    messages.extend(perform_tool_calls(tools=[exa_search], tool_calls=response.tool_calls))
    
    # prompt the model to be grounded
    messages.append({"role": "user","content": "Answer my previous query based on the search results.",})

    final_response = call_model(model_name=MODEL_SMALL, messages=messages)
    return final_response.content

In [35]:
result = research("What are the most popular pokemons?")
console.md(result)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "most popular Pokémon of all time", "num_results": 5}                                           │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The most popular Pokémon vary depending on the source and criteria (e.g., fan votes, trading card game prices, or  
global surveys), but several consistently rank at the top. Based on multiple surveys and fan engagement:           

                                             Top Most Popular Pokémon:                                             

 1 Charizard                                                                                                       
    • Frequently ranked #1 in popularity polls, including a 2019 Reddit survey of over 52,000 fans where it        
      received the highest number of votes.                                                                        
    • Known for its iconic design, role in the anime, and status in the Trading Card Game (TCG).                   
    • Source: ]8;id=228672;https://www.businessinsider.com/most-popular-pokemon-fan-vote-reddit-survey-2019-6\Business Insider]8;;\                                                                                     
 2 Gengar                                                                                                          
    • Ranked #2 in the same Reddit survey with 1,056 votes.                                                        
    • A fan-favorite Ghost-type known for its mischievous personality and strong presence in both games and TCG.   
    • Source: ]8;id=566619;https://www.businessinsider.com/most-popular-pokemon-fan-vote-reddit-survey-2019-6\Business Insider]8;;\                                                                                     
 3 Arcanine                                                                                                        
    • Ranked #3 in the Reddit poll with 923 votes.                                                                 
    • Loved for its noble appearance and powerful stats, often considered one of the most iconic non-legendary     
      Pokémon.                                                                                                     
    • Source: ]8;id=153791;https://www.businessinsider.com/most-popular-pokemon-fan-vote-reddit-survey-2019-6\Business Insider]8;;\                                                                                     
 4 Bulbasaur                                                                                                       
    • Ranked #4 in the Reddit survey with 710 votes.                                                               
    • As one of the original starter Pokémon, it has enduring appeal and strong nostalgic value.                   
    • Source: ]8;id=602720;https://www.businessinsider.com/most-popular-pokemon-fan-vote-reddit-survey-2019-6\Business Insider]8;;\                                                                                     
 5 Eevee                                                                                                           
    • Ranked #9 in the Reddit survey but consistently appears in top lists due to its multiple evolutions          
      ("Eeveelutions").                                                                                            
    • Extremely popular in media, merchandise, and games.                                                          
    • Source: ]8;id=928266;https://www.businessinsider.com/most-popular-pokemon-fan-vote-reddit-survey-2019-6\Business Insider]8;;\                                                                                     

                                              Other Notable Mentions:                                              

 • Umbreon – Often cited in top 10 lists, especially among fans of Dark-type Pokémon.                              
 • Lucario – Ranked #2 in the official Pokémon World Vote with over 102,000 votes.                                 
 • Greninja – Won the official Pokémon World Vote with 140,559 votes, making it the most-voted Pokémon in t

![](images/04_pokedex.png)

This is "Almost" an agent, but it's missing the loop. Let's add that next.

## 4. Implementing a Basic Agentic Loop

Let's implement a basic agentic loop. We'll use the `pokedex` function we just created. The implementation we have above has some limitations:
- Its a single turn, so if it fails to answer my question in one pass it is over.

![](images/05_agent.png)

From the really good [Anthropic Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents) article and encourage people to read it.

A simple for loop

In [36]:
@weave.op
def research_loop(query: str, max_turns: int = 4, tools = [exa_search, add_numbers]) -> str:
    messages = [
        {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format."},
        {"role": "user", "content": query}]
    
    for turn in range(max_turns):
        console.rule(f"Agent Loop Turn {turn + 1}/{max_turns}")

        # call model with tools
        response = call_model(
            model_name=MODEL_MEDIUM, 
            messages=messages, 
            tools=[t.tool_schema for t in tools])

        # add the response to the messages
        messages.append(response.model_dump())

        # if the LLM requested tool calls, perform them
        if response.tool_calls:
            # perform the tool calls
            tool_outputs = perform_tool_calls(tools=[exa_search, add_numbers], tool_calls=response.tool_calls)
            messages.extend(tool_outputs)
        # LLM gave content response
        elif response.content:
            console.rule("Final Model Response")
            console.md(response.content)
            return response.content
        else:
            print("LLM response had neither content nor tool calls. Stopping loop.")
            break

In [37]:
res = research_loop("What is the sum of the populations of the 2 major EU cities?")

─────────────────────────────────────────────── Agent Loop Turn 1/4 ───────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "largest cities by population European Union 2024"}                                             │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────── Agent Loop Turn 2/4 ───────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: add_numbers                                                                                       │
│                                                                                                                 │
│ Args: {"a": 3897145, "b": 3460771}                                                                              │
│                                                                                                                 │
│ ✓ add_numbers executed successfully                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────── Agent Loop Turn 3/4 ───────────────────────────────────────────────

────────────────────────────────────────────── Final Model Response ───────────────────────────────────────────────

Based on the search results from Wikipedia's list of cities in the European Union by population within city limits,
the 2 major EU cities with the largest populations are:                                                            


                                          Largest EU Cities by Population                                          

 1 Berlin, Germany - 3,897,145 (as of December 31, 2024)                                                           
 2 Madrid, Spain - 3,460,771 (as of January 1, 2024)                                                               


                                                Sum of Populations                                                 

The sum of the populations of these 2 major EU cities is: 3,897,145 + 3,460,771 = 7,357,916                        

Source: ]8;id=550811;https://en.wikipedia.org/wiki/List_of_cities_in_the_European_Union_by_population_within_city_limits\List of cities in the European Union by population within city limits - Wikipedia]8;;\                          

Note: These figures represent population within city administrative boundaries rather than metropolitan areas,     
which would typically show larger numbers for cities like Paris and London. However, since the question            
specifically asks about EU cities and the most reliable current data shows Berlin and Madrid as the two largest by 
city proper population, these are the appropriate cities to use for this calculation.

# 5. Structuring the Agent with Classes

The loop above works, but for more complex agents, encapsulating the logic and state within classes is much better. We'll define:
- `AgentState`: A Pydantic model to hold the conversation history and potentially other state.
- `SimpleAgent`: A class containing the agent's configuration (model, system message, tools) and logic (`step`, `run`).

In [38]:
class AgentState(BaseModel):
    """Manages the state of the agent."""
    messages: List[Dict[str, Any]] = Field(default_factory=list)
    step: int = Field(default=0)
    final_assistant_content: str | None = None # Populated at the end of a run

In [39]:
class SimpleAgent:
    """A simple agent class with tracing, state, and tool processing."""
    def __init__(self, model_name: str, system_message: str, tools: List[Callable]):
        self.model_name = model_name
        self.system_message = system_message
        self.tools = [function_tool(t) for t in tools] # add schemas to the tools
    
    @weave.op(name="SimpleAgent.step") # Trace each step
    def step(self, state: AgentState) -> AgentState:
        step = state.step + 1
        messages = state.messages
        final_assistant_content = None
        try:
            # call model with tools
            response = call_model(
                model_name=self.model_name, 
                messages=messages, 
                tools=[t.tool_schema for t in self.tools])

            # add the response to the messages
            messages.append(response.model_dump())

            # if the LLM requested tool calls, perform them
            if response.tool_calls:
                # perform the tool calls
                tool_outputs = perform_tool_calls(tools=self.tools, tool_calls=response.tool_calls)
                messages.extend(tool_outputs)

            # LLM gave content response
            else:
                messages.append(response.model_dump())
                final_assistant_content = response.content
        except Exception as e:
            console.print(f"ERROR in Agent Step: {e}")
            # Add an error message to history to indicate failure
            messages.append({"role": "assistant", "content": f"Agent error in step: {str(e)}"})
            final_assistant_content = f"Agent error in step {step}: {str(e)}"
        return AgentState(messages=messages, step=step, final_assistant_content=final_assistant_content)

    @weave.op(name="SimpleAgent.run")
    def run(self, user_prompt: str, max_turns: int = 10) -> AgentState:
        state = AgentState(messages=[
            {"role": "system", "content": self.system_message},
            {"role": "user", "content": user_prompt}])
        for _ in range(max_turns):
            console.rule(f"Agent Loop Turn {state.step+1}/{max_turns}")
            state = self.step(state)
            if state.final_assistant_content:
                return state
        return state


![](images/07_simple_agent.png)

In [40]:
agent = SimpleAgent(
    model_name=MODEL_SMALL,
    system_message="You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format.",
    tools=[exa_search, add_numbers]
)
state = agent.run(user_prompt="What is the combined weight of Ash's first 3 pokemons?")
print(f"Final response: {state.final_assistant_content}")


────────────────────────────────────────────── Agent Loop Turn 1/10 ───────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "Ash's first 3 Pokémon and their weights", "num_results": 5}                                    │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────── Agent Loop Turn 2/10 ───────────────────────────────────────────────

Final response: Based on the search results, Ash's first three Pokémon are **Bulbasaur**, **Pikachu**, and **Charmander**. Their weights are:

- **Bulbasaur**: 15.2 lbs (6.9 kg)  
- **Pikachu**: 13.2 lbs (6.0 kg)  
- **Charmander**: 18.7 lbs (8.5 kg)  

Adding these weights together:
- 15.2 + 13.2 + 18.7 = **47.1 lbs**  
- 6.9 + 6.0 + 8.5 = **21.4 kg**

Thus, the combined weight of Ash's first three Pokémon is **47.1 lbs (21.4 kg)**.


In [43]:
@weave.op 
@function_tool # <- we can use the decorator to automatically generate the tool schema
def exa_search_and_refine(query: str, num_results: int = 5) -> list[dict[str, str]]:
    """Perform a search query on the web and retrieve the most relevant URLs and web content.
    
    This function uses the Exa search API to find relevant web pages based on the query
    and returns their titles, text content, and URLs.
    
    Args:
        query: The search query. Use detailed, specific queries for better results.
               The quality of results depends on the specificity of the query.
        num_results: The number of search results to retrieve. Defaults to 5.
    
    Returns:
        A list of dictionaries, each containing:
            - title: The title of the web page
            - text: The text content of the web page
            - url: The URL of the web page
    """
    search_results = exa_client.search_and_contents(query=query, type='auto', num_results=num_results)
    
    @weave.op
    def refine_search_result(result, query):
        messages = [
            {"role":"system", "content": f"Your task is to extract from the search results only the info that is relevant to answer the query"},
            {"role": "user", "content": f"- query: {query}\n- Search result: {result}"}
        ]
        refined_search = call_model(model_name=MODEL_SMALL, messages=messages)
        return refined_search.content

    output = []
    for item, result in enumerate(search_results.results):
        console.print(f"Refining result {item+1}")
        refined_text = refine_search_result(result.text, query)
        output.append(
            {"title": result.title,
            "text": refined_text,
            "url": result.url
            }
        )
    return output

In [44]:
agent = SimpleAgent(
    model_name=MODEL_SMALL,
    system_message="You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format.",
    tools=[exa_search_and_refine, add_numbers]
)
state = agent.run(user_prompt="What is the combined weight of Ash's first 3 pokemons?")
print(f"Final response: {state.final_assistant_content}")

────────────────────────────────────────────── Agent Loop Turn 1/10 ───────────────────────────────────────────────

Refining result 1

Refining result 2

Refining result 3

Refining result 4

Refining result 5

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search_and_refine                                                                             │
│                                                                                                                 │
│ Args: {"query": "Ash Ketchum first three Pokémon names and weights", "num_results": 5}                          │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────── Agent Loop Turn 2/10 ───────────────────────────────────────────────

Final response: Ash Ketchum's first three Pokémon, based on canonical sources like Bulbapedia and the Pokémon Wiki, are:

1. **Pikachu** – Weight: **6.0 kg (13.2 lbs)**  
2. **Caterpie** – Weight: **2.9 kg (6.4 lbs)**  
3. **Pidgeotto** – Weight: **30.0 kg (66.1 lbs)**  

### Combined Weight:
- **Metric**: $6.0 + 2.9 + 30.0 = \textbf{38.9 kg}$
- **Imperial**: $13.2 + 6.4 + 66.1 = \textbf{85.7 lbs}$

✅ **Final Answer**: The combined weight of Ash's first three Pokémon is **38.9 kg (85.7 lbs)**.

**Sources**:
- [Bulbapedia - Ash Ketchum](https://bulbapedia.bulbagarden.net/wiki/Ash_Ketchum)
- [Pokémon Wiki - Ash Ketchum/The Beginning](https://pokemon.fandom.com/wiki/Ash_Ketchum/The_Beginning)


Possible improvements to the SimpleAgent:
- Give the model info about the state of the conversation, you could inject a message with the model context pressure, steps left, etc.
- Structured output. Make the model output a specific format, for instance a JSON with the expected fields.
- Add more tools like read and write files, access a database.
- Agent handoff: Agent1 does triage and Agent2 executes specific tasks.